In [0]:
import pandas as pd
import requests
from google.colab import files
from bs4 import BeautifulSoup
import time


# ソートしたいcategoryを選択
category = {0: 'アート/デザイン',
            1: 'インテリア',
            2: 'ガジェット',
            3: 'グルメ/キッチン',
            4: 'ゲーム',
            5: 'ファッション',
            6: 'フォト',
            7: 'ミュージック',
            8: '動画/ムービー',
            9: '広告',
            10: '建築/スポット',
            11: '本',
            12: '雑貨'
           }

# ソートする際に引っ張ってくるURLに組み合わせるtag名
category_url = {0: 'art-design',
                1: 'interior',
                2: 'gadgets',
                3: 'gourmet',
                4: 'games',
                5: 'fashion',
                6: 'photo',
                7: 'music',
                8: 'video',
                9: 'ad',
                10: 'architecture',
                11: 'books',
                12: 'groceries'
               }

print(category)
a = int(input('見たいカテゴリーを選んでください！'))
print(category[a] + 'ですね。ソートして表示します')

# スクレイプするサイトの指定
base_url = 'https://design.style4.info/category/'

# BeautifulSoupで解析＆カテゴリソート済の全ページ数と最終ページ数の取得
res = BeautifulSoup(requests.get(base_url + category_url[a]).text, 'html.parser')
all_pages = res.find('div', {'class': 'pagination clearfix'}).span.text
max_page = int(all_pages.replace('Page 1 of ', ''))
print('ページ数は' + str(max_page) + 'です')

# データフレーム指定
cols = ['item', 'url']
df = pd.DataFrame(columns = cols)

# ここから本番
# 「タイトルとURL取得→dfに入れる」を最終ページまで繰り返し処理
for i in range(1, max_page + 1):
    print('-' * 70)
    
    # 文字化け防止とデータ解析
    response = requests.get(base_url + category_url[a] + '/page/' + str(i))
    response.encoding = response.apparent_encoding
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 記事の１つのカタマリをリスト化でもってくる
    item_list = soup.find_all('div', {'class': 'item-main'})
    # 記事のカタマリを１つずつ持ってくる
    for item in item_list:
        
        # GoogleAdsの回避
        if item.a is None:
            pass
        else:
            url = item.h2.a.get('href')
            name = item.h2.a.string
            print(name)
            print(url)
            se = pd.Series([name, url], cols)
            df = df.append(se, ignore_index=True)
    time.sleep(1)

df.info()
filename = 'Fun_article.csv'
df.to_csv(filename, encoding = 'utf-8-sig')
files.download(filename)
